# utils 

> utils for this project

In [ ]:
# | default_exp utils

In [ ]:
# | hide
# from nbdev.showdoc import *

In [ ]:
# | export

import datetime

import requests

In [ ]:
# | export
def foo():
    pass

In [ ]:
# | hide

nbdev.nbdev_export()

import datetime

import nbdev
import requests